In [ ]:
mixed_raster = '/home/jasp/autocls_exp3/refpxgroups_lvl3_oldid.tif'
db = 'gthruth_valeflores_v3'

pxtbl = 'gthruth_fishnet'

pxtbl_pk = 'cid'

levels = 4

epsg = 32629

outpxgrid = '/home/jasp/autocls_exp3/pxs'
oname = 'px_sample'

In [ ]:
import numpy as np
import datetime as dt
import os 

from glass.prop.prj import get_epsg
from glass.prop.sql import check_last_id
from glass.rd.rst import rst_to_array
from glass.sql.q import q_to_obj
from glass.wt.shp import df_to_shp
from glass.wt.sql import df_to_db
from glass.smp.fish.sql import split_fishnetcells_in_four

In [ ]:
time_a = dt.datetime.now().replace(microsecond=0)

last_id = check_last_id(db, pxtbl_pk, pxtbl)

for i in range(1, levels):
    subfish = split_fishnetcells_in_four(
        db, pxtbl, {'cid' : 'rootid'},
        'geom', epsg, last_id, 'cid', 'cid',
        whr=f"{pxtbl}.level = {str(i)}",
        valascol={"level" : str(i+1)}
    )

    last_id = subfish.cid.max()

    df_to_shp(subfish, os.path.join(outpxgrid, f'{oname}_{str(i + 1)}.shp'))
    df_to_db(
        db, subfish, pxtbl,
        append=True, col_geom="geom",
        epsg=epsg
    )

    time_b = dt.datetime.now().replace(microsecond=0)

In [ ]:
filter = (
        "SELECT cid, "
        "ST_XMin(geom) AS xleft, ST_XMax(geom) AS xright, "
        "ST_YMin(geom) AS ybottom, ST_YMax(geom) AS ytop, "
        "(ST_XMax(geom) - ST_XMin(geom)) / 2 AS xwidth, "
        "(ST_YMax(geom) - ST_YMin(geom)) / 2 AS yheight "
        f"FROM {pxtbl} AS tbl "
        f"WHERE tbl.level = {str(i)}"
        #"WHERE  cid = ANY(ARRAY[{whra}])"
    )

    q = (
        "SELECT cid, "
        "ST_SetSRID(ST_Collect(ARRAY["
            "ST_MakePolygon(ST_MakeLine(ARRAY["
                "ST_MakePoint(xleft, ytop), "
            	"ST_MakePoint(xleft, ytop - yheight), "
            	"ST_MakePoint(xleft + xwidth, ytop - yheight), "
            	"ST_MakePoint(xleft + xwidth, ytop), "
            	"ST_MakePoint(xleft, ytop)"
            "])), "
            "ST_MakePolygon(ST_MakeLine(ARRAY["
            	"ST_MakePoint(xleft + xwidth, ytop),"
            	"ST_MakePoint(xleft + xwidth, ytop - yheight), "
            	"ST_MakePoint(xright, ytop - yheight), "
            	"ST_MakePoint(xright, ytop), "
            	"ST_MakePoint(xleft + xwidth, ytop)"
            "])),"
            "ST_MakePolygon(ST_MakeLine(ARRAY["
            	"ST_MakePoint(xleft, ytop - yheight), "
            	"ST_MakePoint(xleft, ybottom), "
            	"ST_MakePoint(xleft + xwidth, ybottom), "
            	"ST_MakePoint(xleft + xwidth, ytop - yheight), "
            	"ST_MakePoint(xleft, ytop - yheight)"
            "])), "
            "ST_MakePolygon(ST_MakeLine(ARRAY["
            	"ST_MakePoint(xleft + xwidth, ytop - yheight), "
            	"ST_MakePoint(xleft + xwidth, ybottom), "
            	"ST_MakePoint(xright, ybottom), "
            	"ST_MakePoint(xright, ytop - yheight), "
            	"ST_MakePoint(xleft + xwidth, ytop - yheight)"
            "]))"
        f"]), {str(epsg)}) AS geom "
        f"FROM ({filter}) AS foo"
    )

    fq = (
        f"SELECT ROW_NUMBER() OVER(ORDER BY rootid) + {str(last_id)} AS cid, "
        f"{str(i + 1)} AS level, * FROM ("
            "SELECT cid AS rootid, "
            f"(ST_Dump(geom)).geom AS geom "
            f"FROM ({q}) AS foo2"
        ") AS foo3"
    )